In [1]:
import pandas as pd

from alethia import alethia

In [2]:
df = pd.read_csv("../data/India_district_mappings.csv")
df

,State,1951,1961,1971,1981,1991,2001
0,Andaman and Nicobar Islands,Andaman and Nicobar,Andaman and Nicobar Islands,Andaman and Nicobar Islands,Andaman and Nicobar Islands,Andaman and Nicobar Islands,Andaman and Nicobar Islands
1,Andaman and Nicobar Islands,Andaman and Nicobar,Andaman and Nicobar Islands,Andaman and Nicobar Islands,Andaman,Andaman,Andaman
2,Andaman and Nicobar Islands,Andaman and Nicobar,Andaman and Nicobar Islands,Andaman and Nicobar Islands,Nicobar,Nicobar,Nicobar
3,Andhra Pradesh,Srikakulam,Srikakulam,Srikakulam,Srikakulam,Srikakulam,Srikakulam
4,Andhra Pradesh,Srikakulam,Srikakulam,Srikakulam,Vizianagaram,Vizianagaram,Vizianagaram
...,...,...,...,...,...,...,...
631,West Bengal,Burdwan,Burdwan,Burdwan,Burdwan,Burdwan,Burdwan
632,West Bengal,Birbhum,Birbhum,Birbhum,Birbhum,Birbhum,Birbhum
633,West Bengal,Bankura,Bankura,Bankura,Bankura,Bankura,Bankura
634,West Bengal,Midnapur,Midnapore,Midnapore,Midnapore,Midnapore,Midnapore


In [3]:
target_list = list(df["2001"])
correct_entries = target_list
reference_entries = [x for x in list(set(target_list)) if str(x) != "nan"]

In [4]:
col = "1951"
incorrect_entries = df[col].tolist()
alethia_output = alethia(incorrect_entries, reference_entries)

Initializing model Salesforce/SFR-Embedding-Mistral
Loading model Salesforce/SFR-Embedding-Mistral


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Successfully loaded model: Salesforce/SFR-Embedding-Mistral
Initial resource usage:
CPU Usage: 4.8%
RAM Usage: 8.6%
GPU: NVIDIA RTX A6000
GPU Memory Usage: 26.49GB / 47.41GB
Computing reference embeddings for 584 entries...
Processing 636 incorrect entries...


Processing entries: 100%|██████████████████| 636/636 [00:37<00:00, 16.81entry/s]

Processing completed in 69.70 seconds


In [5]:
alethia_output[alethia_output.given_entity != alethia_output.alethia_prediction]

,given_entity,alethia_prediction,alethia_score
0,Andaman and Nicobar,Andaman and Nicobar Islands,0.994246
1,Andaman and Nicobar,Andaman and Nicobar Islands,0.994246
2,Andaman and Nicobar,Andaman and Nicobar Islands,0.994246
5,Vizagapatnam,Visakhapatnam,0.969572
6,Vizagapatnam,Visakhapatnam,0.969572
...,...,...,...
622,West Dinajpur,Uttar Dinajpur,0.941147
626,24 - Parganas,South Twenty Four Parganas,0.928405
627,24 - Parganas,South Twenty Four Parganas,0.928405
634,Midnapur,Midnapore,0.954628


# Plotting examples

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from alethia import (
    do_pca,
    do_umap,
    get_embeddings,
    load_sentence_transformer,
    plot_embedding,
    setup_matplotlib,
)
setup_matplotlib()

In [ ]:
model = load_sentence_transformer("Salesforce/SFR-Embedding-Mistral")

Loading model Salesforce/SFR-Embedding-Mistral


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df = pd.read_csv("../data/India_district_mappings.csv")
df

In [ ]:
india_districts = pd.read_csv("../data/India_all_state_districts.csv")
districts_to_keep = india_districts.dtname.value_counts()
districts_to_keep = districts_to_keep[districts_to_keep < 2]

reference_entries = set(india_districts["dtname"])
reference_entries = districts_to_keep.index.tolist()
india_districts_reduced = india_districts[
    india_districts.dtname.isin(districts_to_keep.index.tolist())
]

palette = (
    list(sns.color_palette("tab20"))
    + list(sns.color_palette("Set3", 12))
    + list(sns.color_palette("Dark2", 8))
)

palette = palette[:36]
valid_df = india_districts_reduced[
    india_districts_reduced["dtname"].isin(reference_entries)
].copy()
unique_states = valid_df["stname"].unique()
color_map = dict(zip(unique_states, palette))
labels = valid_df["dtname"].tolist()
states = valid_df["stname"].tolist()

In [ ]:
embedding = get_embeddings(
    texts=labels, model=model, return_labels=False, show_progress=True
)

In [ ]:
pca = do_pca(embedding)

In [ ]:
pca, exp_var = do_pca(embedding, return_expl_var=True)

In [ ]:
umap = do_umap(embedding)

In [ ]:
plt.figure(figsize=(10, 8))
plot_embedding(
    pca, labels=states, dims=[1, 2], color_map=palette, title="", explained_var=exp_var
)

In [ ]:
plt.figure(figsize=(10, 8))
plot_embedding(umap, labels=states, dims=[1, 2], color_map=palette, title="")